À partir des datasets `population_communes.csv` et `surface_departements.csv`, créer un nouveau dataset qui contient une ligne par département, avec ces colonnes:

- la somme des "Population municipale" du département
- la somme des "Population totale" du département (pour l'explication de la distinction entre "Population municipale" et "Population totale", voir: https://www.insee.fr/fr/metadonnees/definition/c1270)
- la part (en pourcentage) de la population municipale par rapport à la population totale
- la part (en pourcentage) de la population (municipale) du département au sein de sa région
- la densité de la population (municipale) en nb d'habitants / km2

(Le dataset final devrait ressembler à `result-exo-cc.csv`)


In [2]:
import pandas as pd

In [34]:
pop_communes = pd.read_csv("population_communes.csv")
surf_dep = pd.read_csv("surface_departements.csv")

In [7]:
surf_dep

,code_insee,nom,surf_km2
0,974,La Réunion,2505.0
1,11,Aude,6343.0
2,43,Haute-Loire,5003.0
3,13,Bouches-du-Rhône,5247.0
4,47,Lot-et-Garonne,5385.0
...,...,...,...
96,56,Morbihan,6870.0
97,25,Doubs,5256.0
98,39,Jura,5049.0
99,07,Ardèche,5566.0


In [8]:
pop_communes.head()

,Code département,Code canton,Code arrondissement,Code région,Nom de la commune,Code commune,Nom de la région,variable,value
0,64,27.0,3,75,Aast,1,Nouvelle-Aquitaine,Population totale,184
1,64,27.0,3,75,Aast,1,Nouvelle-Aquitaine,Population municipale,177
2,55,10.0,2,44,Abainville,1,Grand Est,Population totale,310
3,55,10.0,2,44,Abainville,1,Grand Est,Population municipale,305
4,60,11.0,1,32,Abancourt,1,Hauts-de-France,Population totale,658


In [35]:
pop_totale = pop_communes.loc[pop_communes["variable"]== "Population totale"][["Code département", "value"]].groupby("Code département").sum()
pop_totale = pop_totale.rename(columns={"value": "population totale"})

In [36]:
pop_mun = pop_communes.loc[pop_communes["variable"]== "Population municipale"][["Code département", "value"]].groupby("Code département").sum()
pop_mun = pop_mun.rename(columns={"value": "population municipale"})

In [37]:
data = surf_dep.set_index('code_insee').join(pop_mun ).join(pop_totale)

In [38]:
data.head()

,nom,surf_km2,population municipale,population totale
code_insee,,,,
974,La Réunion,2505.0,852924.0,862814.0
11,Aude,6343.0,368025.0,377580.0
43,Haute-Loire,5003.0,227339.0,234613.0
13,Bouches-du-Rhône,5247.0,2019717.0,2047433.0
47,Lot-et-Garonne,5385.0,332833.0,342358.0


In [42]:
data['part pop mun / pop totale'] = data['population municipale'].div(data['population totale']/100)

In [43]:
data.head()

,nom,surf_km2,population municipale,population totale,part pop mun / pop totale
code_insee,,,,,
974,La Réunion,2505.0,852924.0,862814.0,98.853751
11,Aude,6343.0,368025.0,377580.0,97.469410
43,Haute-Loire,5003.0,227339.0,234613.0,96.899575
13,Bouches-du-Rhône,5247.0,2019717.0,2047433.0,98.646305
47,Lot-et-Garonne,5385.0,332833.0,342358.0,97.217825


In [94]:
pop_dep_reg = pop_communes.loc[pop_communes["variable"]== "Population municipale"][["Code région","Code département", "value"]].groupby(["Code région", "Code département"]).sum()
data = data.join(pop_dep_reg.groupby('Code région').transform('sum').reset_index().set_index('Code département')['value'])
data['pct pop dep / region'] = data['population municipale'] / data['value'] *100

In [73]:
pop_communes

,Code département,Code canton,Code arrondissement,Code région,Nom de la commune,Code commune,Nom de la région,variable,value
0,64,27.0,3,75,Aast,1,Nouvelle-Aquitaine,Population totale,184
1,64,27.0,3,75,Aast,1,Nouvelle-Aquitaine,Population municipale,177
2,55,10.0,2,44,Abainville,1,Grand Est,Population totale,310
3,55,10.0,2,44,Abainville,1,Grand Est,Population municipale,305
4,60,11.0,1,32,Abancourt,1,Hauts-de-France,Population totale,658
...,...,...,...,...,...,...,...,...,...
70759,62,38.0,1,32,Œuf-en-Ternois,633,Hauts-de-France,Population municipale,251
70760,51,6.0,2,44,Œuilly,410,Grand Est,Population municipale,635
70761,02,6.0,2,32,Œuilly,565,Hauts-de-France,Population totale,292
70762,51,6.0,2,44,Œuilly,410,Grand Est,Population totale,645


In [97]:
data['densité pop'] = data['population municipale'].div(data['surf_km2'])
data = data.drop(columns=["surf_km2", "value"])

In [98]:
data.head()

,nom,population municipale,population totale,part pop mun / pop totale,densité pop,pct pop dep / region
code_insee,,,,,,
974,La Réunion,852924.0,862814.0,98.853751,340.488623,100.000000
11,Aude,368025.0,377580.0,97.469410,58.020653,6.336044
43,Haute-Loire,227339.0,234613.0,96.899575,45.440536,2.871570
13,Bouches-du-Rhône,2019717.0,2047433.0,98.646305,384.927959,40.217960
47,Lot-et-Garonne,332833.0,342358.0,97.217825,61.807428,5.607400


In [ ]:
df.columns.name = None # remove index column name 